In [1]:
# librerías
import os
import pandas as pd
import gower
from sklearn.svm import SVC
from sklearn.model_selection import  GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
from sklearn.manifold import MDS
import gc




In [3]:
# directorios
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')
image_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'images')


In [11]:
# dataset a (edades categorizadas)

dataset_a= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'))

In [15]:
# categorizar edad en dataset a


def categoria_edad (x):
    if (x >= 0) and (x <= 11) :
        return 'Niñez'
    elif (x >= 12) and (x <=18):
        return 'Adolescencia'
    elif (x >= 19) and (x <=30):
        return 'Juventud'
    elif (x>=31) and (x<=65) :
        return 'Vejez'
    elif x>=66:
        return 'Vejez mayor'
    else:
        return 'NS/NC'


dataset_a['victima_edad_cat'] = \
dataset_a.victima_edad.apply(categoria_edad)
dataset_a['llamante_edad_cat'] = \
dataset_a.llamante_edad.apply(categoria_edad)

In [16]:
# drop columnas sin usar

dataset_a.drop(['victima_edad', 'llamante_edad'], axis=1, inplace=True) 

In [17]:
# reemplazar nsnc por na
dataset_a.loc[:, 'victima_convive_agresor'] = dataset_a['victima_convive_agresor'].replace({'NS/NC': pd.NA})


In [20]:
# mapear SI NO a 1 0
dataset_a['victima_convive_agresor'] = dataset_a['victima_convive_agresor'].map({'SI': 1, 'NO':0})

In [27]:
# chequear proporciones previas
print('prop. SI:', len(dataset_a[dataset_a['victima_convive_agresor']==1])/len(dataset_a)*100)
print('prop. NO:',len(dataset_a[dataset_a['victima_convive_agresor']==0])/len(dataset_a)*100)

prop. SI: 14.39690748576503
prop. NO: 64.40474324818472


In [21]:
# separar features de target
X = dataset_a.drop(['victima_convive_agresor'], axis=1)
y_previo = dataset_a['victima_convive_agresor']

In [28]:
# guardar los índices de casos NSNC (vacíos)
nsnc_indices = y_previo[y_previo.isna()].index

In [29]:
# Remove "NSNC" rows from the target
y = y_previo.drop(nsnc_indices)

In [30]:
# gower de X
gower_X = gower.gower_matrix(X)
print("gower para dataset_a hecho")

# correr NMDS sobre el total del dataset
nmds = MDS(n_components=3 ,metric=False, dissimilarity='precomputed', max_iter=300, random_state=0, normalized_stress=True) 
X_nmds = nmds.fit_transform(gower_X)
print('Fit transform hecho')
# crear el test final con lo que corresponde a target de X transformado 
test_final = X_nmds[nsnc_indices]
print('Test final hecho')

print('Stress: ' + str(round(nmds.stress_,2)))
# quitar el test final 
X_nmds_clean = pd.DataFrame(X_nmds).drop(nsnc_indices)


print('entrenando el modelo svm')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2)
for train_index, test_index in sss.split(X_nmds_clean, y):
    X_train, X_test = X_nmds_clean.iloc[train_index], X_nmds_clean.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Parámetros de gridsearch
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}


svm = SVC(class_weight='balanced')


# GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_

# Aplicar al test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)
print("Test set accuracy: ", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))


########################################################

gc.collect()


gower para dataset_a hecho
Fit transform hecho
Test final hecho
Stress: 0.25
entrenando el modelo svm
Best parameters found:  {'C': 0.1, 'gamma': 'auto', 'kernel': 'poly'}
Best cross-validation accuracy:  0.8172853753024473
Test set accuracy:  0.8173682466025853
Classification report:
               precision    recall  f1-score   support

         0.0       0.82      1.00      0.90      2466
         1.0       0.00      0.00      0.00       551

    accuracy                           0.82      3017
   macro avg       0.41      0.50      0.45      3017
weighted avg       0.67      0.82      0.74      3017



/Users/vcolombo/Documents/tp especializacion/tpenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vcolombo/Documents/tp especializacion/tpenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vcolombo/Documents/tp especializacion/tpenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

25

### SVM con nmds sobre dataset a con n_components= 2
- Best parameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
- Best cross-validation accuracy:  0.8172853753024473
- Test set accuracy:  0.8173682466025853


Classification report:

- precision: 0.82(NO) 0 (SI)
- recall: 1.0(NO) 0 (SI) 
- f1-score: 0.90(NO) 0 (SI)
- support: 2466(NO) 551 (SI)


- accuracy 0.82      3017
- macro avg       (p)0.41     (r) 0.50     (f1) 0.45    (supp)  3017
- weighted avg     (p)  0.67   (r)   0.82  (f1)    0.74 (supp)     3017

In [2]:
# Apply the best model to the subset of the original dataset
#na_predictions = best_model.predict(test_final)

# Add the predictions to the na_convive_df DataFrame
#test_final['victima_convive_agresor_pred'] = na_predictions

#print("\nPredictions for NA 'convive' values:")
#print(test_final)

NameError: name 'best_model' is not defined

In [ ]:
# dataset b (sin llamante_edad y con casos completos de victima_edad)

llamados_v2= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'))

In [ ]:
# quitar llamante edad
llamados_v2.drop(['llamante_edad'], axis=1, inplace=True) 

In [ ]:
# descartar vacíos de victima edad
dataset_b = llamados_v2[~(llamados_v2['victima_edad'].isnull())]

In [ ]:
# reemplazar nsnc por na
dataset_b.loc[:, 'victima_convive_agresor'] = dataset_b['victima_convive_agresor'].replace({'NS/NC': pd.NA})

In [ ]:
# mapear SI NO a 1 0
dataset_b['victima_convive_agresor'] = dataset_b['victima_convive_agresor'].map({'SI': 1, 'NO':0})

In [ ]:
# chequear proporciones previas
print('prop. SI:', len(dataset_b[dataset_b['victima_convive_agresor']==1])/len(dataset_b)*100)
print('prop. NO:',len(dataset_b[dataset_b['victima_convive_agresor']==0])/len(dataset_b)*100)

In [ ]:
# separar features de target
X = dataset_b.drop(['victima_convive_agresor'], axis=1)
y_previo = dataset_b['victima_convive_agresor']

In [ ]:
# guardar los índices de casos NSNC (vacíos)
nsnc_indices = y_previo[y_previo.isna()].index

In [ ]:
# Remove "NSNC" rows from the target
y = y_previo.drop(nsnc_indices)

In [ ]:
# gower de X
gower_X = gower.gower_matrix(X)
print("gower para dataset_a hecho")

# correr NMDS sobre el total del dataset
nmds = MDS(n_components=2 ,metric=False, dissimilarity='precomputed', max_iter=300, random_state=0, normalized_stress=True) 
X_nmds = nmds.fit_transform(gower_X)
print('Fit transform hecho')
# crear el test final con lo que corresponde a target de X transformado 
test_final = X_nmds[nsnc_indices]
print('Test final hecho')

print('Stress: ' + str(round(nmds.stress_,2)))
# quitar el test final 
X_nmds_clean = pd.DataFrame(X_nmds).drop(nsnc_indices)


print('entrenando el modelo svm')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2)
for train_index, test_index in sss.split(X_nmds_clean, y):
    X_train, X_test = X_nmds_clean.iloc[train_index], X_nmds_clean.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Parámetros de gridsearch
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}


svm = SVC(class_weight='balanced')


# GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_

# Aplicar al test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)
print("Test set accuracy: ", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))


########################################################

gc.collect()
